# システムプロンプト

In [ ]:
system_prompt = '''
You are a helpful assistant that helps people find information.Please ensure the output is always in Markdown format.
'''

# プロンプト

In [ ]:
prompt = '''
こんにちは
'''

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()  # take environment variables

from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import httpx
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    azure_ad_token_provider=token_provider,
    timeout=httpx.Timeout(3600.0, read=3600.0, write=3600.0, connect=3600.0)
)


In [ ]:
system_prompt = system_prompt.replace("\\n","\n")
prompt = prompt.replace("\\n","\n").replace("\\t","\t") + "ただし、出力はMarkdownの形式で出力すること。"

In [ ]:
response = client.chat.completions.create(
    model=os.environ["OPENAI_MODEL_NAME"],
    messages=[
        {
            "role": "system",
            "content": f"{system_prompt}"
        },
        {
            "role": "user",
            "content": [ {
                "type": "text",
                "text": prompt
            }]
        }
    ]
)

In [ ]:
print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)
print("\tTotal tokens:", response.usage.total_tokens)

In [ ]:
with open(f"output-{os.environ['OPENAI_MODEL_NAME']}.markdown", "w") as o:
    print(response.choices[0].message.content, file=o)